In [1]:
import pandas as pd
data = pd.read_csv('energydata_complete.csv')

In [15]:
!python -m pip install sklearn

Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'


In [13]:
data.head()

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,50,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,50,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,60,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [12]:
data.drop('date',axis=1,inplace=True)
data.drop('lights',axis=1,inplace=True)

In [59]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
features_df = normalised_df.drop(columns='Appliances')

target = normalised_df['Appliances']
print(features_df)
print(target)

             T1      RH_1        T2      RH_2        T3      RH_3        T4  \
0      0.327350  0.566187  0.225345  0.684038  0.215188  0.746066  0.351351   
1      0.327350  0.541326  0.225345  0.682140  0.215188  0.748871  0.351351   
2      0.327350  0.530502  0.225345  0.679445  0.215188  0.755569  0.344745   
3      0.327350  0.524080  0.225345  0.678414  0.215188  0.758685  0.341441   
4      0.327350  0.531419  0.225345  0.676727  0.215188  0.758685  0.341441   
...         ...       ...       ...       ...       ...       ...       ...   
19730  0.926786  0.537657  0.711655  0.606309  0.830841  0.579374  0.864865   
19731  0.919747  0.536006  0.701769  0.607836  0.825302  0.582178  0.864865   
19732  0.919747  0.538666  0.692651  0.627198  0.818378  0.603988  0.864865   
19733  0.919747  0.549491  0.677054  0.634717  0.805085  0.585294  0.864865   
19734  0.919747  0.538758  0.666171  0.632902  0.799546  0.579062  0.864865   

           RH_4        T5      RH_5  ...        T9 

In [60]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features_df, target,
test_size=0.3, random_state=42)

from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()

linear_model.fit(x_train, y_train)

predicted_values = linear_model.predict(x_test)


In [61]:





from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
print(round(mae, 3) )
# Residual Sum of Squares (RSS)
import numpy as np
rss = np.sum(np.square(y_test - predicted_values))
print(round(rss, 3) )
# Root Mean Square Error (RMSE)
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
print(round(rmse, 3) )
# R-Squared
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
print(round(r2_score, 3) )

0.05
45.345
0.088
0.149


In [64]:
linear_model.coef_

array([-3.28105119e-03,  5.53572379e-01, -2.36124035e-01, -4.56613168e-01,
        2.90551044e-01,  9.59883569e-02,  2.90002666e-02,  2.64009881e-02,
       -1.56592007e-02,  1.59972570e-02,  2.36384782e-01,  3.80571628e-02,
        1.03241380e-02, -4.46237001e-02,  1.01995153e-01, -1.57595860e-01,
       -1.89921236e-01, -3.98011119e-02, -3.21795827e-01,  6.83875095e-03,
       -7.76504670e-02,  2.91870886e-02,  1.23061409e-02,  1.17720893e-01,
        1.01553273e+11, -1.01553273e+11])

In [70]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)

from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

def get_weights_df(linear_model, feat, col_name):
    weights = pd.Series(linear_model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df
linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')


In [71]:
print(linear_model_weights)
print(ridge_weights_df)
print(lasso_weights_df)
print(final_weights)
print(final_weights)

       Features  Linear_Model_Weight
0           rv2        -1.015533e+11
1          RH_2        -4.566132e-01
2         T_out        -3.217958e-01
3            T2        -2.361240e-01
4            T9        -1.899212e-01
5          RH_8        -1.575959e-01
6        RH_out        -7.765047e-02
7          RH_7        -4.462370e-02
8          RH_9        -3.980111e-02
9            T5        -1.565920e-02
10           T1        -3.281051e-03
11  Press_mm_hg         6.838751e-03
12           T7         1.032414e-02
13   Visibility         1.230614e-02
14         RH_5         1.599726e-02
15         RH_4         2.640099e-02
16           T4         2.900027e-02
17    Windspeed         2.918709e-02
18         RH_6         3.805716e-02
19         RH_3         9.598836e-02
20           T8         1.019952e-01
21    Tdewpoint         1.177209e-01
22           T6         2.363848e-01
23           T3         2.905510e-01
24         RH_1         5.535724e-01
25          rv1         1.015533e+11
 

In [73]:
count=0
for x in lasso_weights_df:
    if x!=0:
        count+=1
print(count)        

2


In [74]:
print(lasso_weights_df)

       Features  Lasso_weight
0        RH_out     -0.049557
1          RH_8     -0.000110
2            T1      0.000000
3     Tdewpoint      0.000000
4    Visibility      0.000000
5   Press_mm_hg     -0.000000
6         T_out      0.000000
7          RH_9     -0.000000
8            T9     -0.000000
9            T8      0.000000
10         RH_7     -0.000000
11          rv1     -0.000000
12           T7     -0.000000
13           T6      0.000000
14         RH_5      0.000000
15           T5     -0.000000
16         RH_4      0.000000
17           T4     -0.000000
18         RH_3      0.000000
19           T3      0.000000
20         RH_2     -0.000000
21           T2      0.000000
22         RH_6     -0.000000
23          rv2     -0.000000
24    Windspeed      0.002912
25         RH_1      0.017880


In [75]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
print(round(rmse, 3) )

0.088
